In [1]:
import json
import requests
from pprint import pprint

In [4]:
# Define the names for the data source, skillset, index and indexer
datasource_name = "handonstorage01"
skillset_name = "cogsrch-py-skillset"
index_name = "cogsrch-py-index"
indexer_name = "cogsrch-py-indexer"

In [5]:
# Setup the endpoint
endpoint = 'https://websitetutorial.search.windows.net'
headers = {'Content-Type': 'application/json',
           'api-key': '56D84BD56C9802F798E2214891F97A8C'}
params = {
    'api-version': '2020-06-30'
}

In [6]:
# Create a data source\n",
datasourceConnectionString = "DefaultEndpointsProtocol=https;AccountName=handonstorage01;AccountKey=rCTSmkICcGjWXi4sX5E6yUEwK9FU0Qr13Q9MDE1pvLoFiR2tKNphP58dvf1+W317IteD3Vipw4vRuCmzKOCuAw==;EndpointSuffix=core.windows.net"
datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": datasourceConnectionString,
    },
    "container": {
        "name": "cog-search-demo",
    },
}
r = requests.put(endpoint+"/datasources/"+datasource_name, data=json.dumps(datasource_payload), headers=headers, params=params )
print(r.status_code)

201


In [7]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description":
    "Extract entities, detect language and extract key-phrases",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Text.EntityRecognitionSkill",
            "categories": ["Organization"],
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/content"
                }
            ],
            "outputs": [
                {
                    "name": "organizations", 
                    "targetName": "organizations"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/content"
                }
            ],
            "outputs": [
                {
                    "name": "languageCode",
                    "targetName": "languageCode"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "textSplitMode": "pages",
            "maximumPageLength": 4000,
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/content"
                },
                {
                    "name": "languageCode",
                    "source": "/document/languageCode"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "pages"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
            "context": "/document/pages/*",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/pages/*"
                },
                {
                    "name": "languageCode", 
                    "source": "/document/languageCode"
                }
            ],
            "outputs": [
                {
                    "name": "keyPhrases",
                    "targetName": "keyPhrases"
                }
            ]
        }
    ]
}

r = requests.put(endpoint + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)

201
